# Python Tutorial

**1.** 声明变量和基本数学运算
> python是一个弱类型解释型语言, 因此声明变量的方式十分简单

In [1]:
a = 10
a

10

In [2]:
b = 100
c = 23

In [3]:
a + b

110

In [4]:
b * c + c * (b - a)

4370

**2.** 声明函数
> python中的声明函数也十分简单, 但是近些年来, python提供了类型标注功能. 这写标注对python解释器毫无影响, 但是可以帮助程序员理解和构建程序, 也可以帮助IDE完成自动补全等功能.

In [5]:
def add(a: int, b: int) -> int:
    return a + b


add(1, 2)  # 正确使用

3

In [6]:
add('Hello ', 'World!') # 这种使用虽然与类型标注不符, 但是最多只会使编译器提示Warning, 并不会造成Error

'Hello World!'

**3.** 高阶函数
> 高阶函数是以其他函数作为参数或返回一个函数的函数
> 高阶函数有许多作用, 比如所有的装饰器(decorator)都是高阶函数, 而高阶函数也可以用来完成一些数据封装之类的任务.

In [7]:
def trace(fun):
    def traced(n):
        print(f"{fun.__name__}({n})")
        return fun(n)

    return traced

In [8]:
@trace
def fact(n):
    if n == 0:
        return 1
    else:
        return n * fact(n - 1)
    
# fact = trace(fact)

In [9]:
fact(5) # 这里奇怪的输出是tuple的问题

fact(5)
fact(4)
fact(3)
fact(2)
fact(1)
fact(0)


120

> 接下来我将展示高阶函数在数据封装上的应用, 一个典型的例子就是WithdrawAccount(一个只能取钱的银行账户)

In [10]:
def makeAccount(holder:str, balance:int):
    def withdraw(amount):
        nonlocal balance
        if amount > balance:
            return 'Insufficient funds'
        balance -= amount
        return balance
    return withdraw

In [11]:
jack = makeAccount('Jack', 100)
rose = makeAccount('Rose', 200)

In [12]:
jack(50)

50

In [13]:
rose(50) # jack和rose是两个不同的闭包, 互不影响

150

In [14]:
jack(100)

'Insufficient funds'

In [15]:
rose(100)

50

**4.** python的计算规则(Evaluation Rules)
> 在3中, 可能会遇到一个问题: `jack`和`rose`的`balance`是如何维护的呢? 换句话说, 怎么保证`jack`不会取出来`rose`的钱呢?
> 这就要展开介绍一下python的environment机制了
---
在python中, 名字(name)和值(value)的绑定是由一个字典来保障的, 每个名字都会跟一个值(类型并不重要)有对应关系. 而这些绑定都是可以随意变换的

In [16]:
max # 此时max这个名字绑定的是一个内置的函数

<function max>

In [17]:
# 可以将max从新绑定到一些别的东西, 比如2
max = 2

In [18]:
# 现在max就是2了
max

2

以上的name和value的binding都是在一个叫做`global frame`的环境中完成的(确切的说, 此时的`environment`里只有`global frame`), 而我们调用一个函数时, 就会在`environment`中加入一个新的`frame`, 这个新的`frame`的`parent`就是这个函数被调用时所在的`frame`. 

听起来有且拗口,下面我们将用一个例子来展示. 

In [19]:
x = 1 # 在global frame中定义了一个x
def f():
    x = 2 # 在f的frame中定义了一个x
    return x

In [20]:
f() # 此时在global frame中调用f, f新创造的frame(称其为f)的parent是global frame, f中的x是2

2

In [21]:
def g():
    x = 3# 这里的x是g的frame中的x
    return x + f() # 这里的f()是调用f函数, 在g frame中创造f frame, f frame中的x是2, g frame中的x是3

In [22]:
g()

5

**由以上可以看出, 在计算一个name绑定的value时, 先会在当前`environment`的`frame`中寻找, 若找到则不会进入之前的`frame`寻找**

In [23]:
def h():
    return x + 4

可以猜测一下调用`h()`会得到什么结果

In [24]:
h()

5

**这是因为, frame `h`的parent是`global`, `h`中并没有对`x`的绑定, 因此在计算`x`的值时, 当前`environment`的`frame`中没有找到这个name, 故不断会进入当前`frame`的上层(也就是其parent), 直到`global`为止. 若是在整个`environment`中都没有找到这个name, 则会报错.**

In [25]:
y # 会报错

NameError: name 'y' is not defined

In [26]:
def fu():
    return y # 会报错

In [29]:
fu()

NameError: name 'y' is not defined